In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit import rdBase
from rdkit.Chem import Draw, AllChem
from IPython.display import display,Image, SVG
from rdkit.Chem import rdmolops
rdBase.DisableLog('rdApp.error')
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from tqdm import tqdm, trange
import pickle
from rdkit import Chem, DataStructs

In [ ]:
import sys
sys.path.append('./scscore')
sys.path.append('./scscore/scscore')

In [ ]:
import os
from rdkit.Chem import RDConfig
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer

In [ ]:
import standalone_model_numpy
from standalone_model_numpy import SCScorer
scscore_model = SCScorer()
scscore_model.restore('./scscore/model.ckpt-10654.as_numpy.json.gz')

In [ ]:
df_generated = pd.read_csv('./results/generative_model/Generated_Practical_Set.csv')
exp_smiles = pd.read_csv('./data/Experimental_Ligands.csv')

In [ ]:
exp_smiles_list = list(exp_smiles['smiles'])
print('Number of real molecules :', len(exp_smiles_list))
df_generated_list = list(df_generated['smiles'])
print('Number of generated molecules :', len(df_generated_list))

In [ ]:
scscore_gen = []
sascore_gen = []

for i in range(len(df_generated_list)):
    sm, score = scscore_model.get_score_from_smi(df_generated_list[i])
    sascore = sascorer.calculateScore(Chem.MolFromSmiles(df_generated_list[i]))
    sascore_gen.append(sascore)
    scscore_gen.append(score)

scscore_real = []
sascore_real = []

for i in range(len(exp_smiles_list)):
    sm, score = scscore_model.get_score_from_smi(exp_smiles_list[i])
    sascore = sascorer.calculateScore(Chem.MolFromSmiles(exp_smiles_list[i]))
    sascore_real.append(sascore)
    scscore_real.append(score)

#scs_plot

#fig, ax = plt.subplots
plt.figure(figsize=(6, 4))
p1 = sns.kdeplot(scscore_real,  fill=True, label=str(np.round(np.mean(scscore_real),2)), linewidth=3,alpha=.5) # 'Experimental = ' +color='red',
p1 = sns.kdeplot(scscore_gen,  fill=True, label=str(np.round(np.mean(scscore_gen),2)), linewidth=3,alpha=.5) #'Generated = ' + color='Green',
#p1.set(xlabel='Synthetic Complexeity Score', title='Distribution of SC Score for generated molecules')
p1.legend(bbox_to_anchor=(1.00, 1.00), loc='best', fontsize=12)
p1.set_xlabel('Synthetic complexeity score', fontsize=14)  # Adjust the font size as needed
p1.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/SCSCORE_experimental_vs_generated.png',dpi = 600)
plt.show()

#sas_plot
#fig2, ax2 = plt.subplots()
plt.figure(figsize=(6, 4))
p2 = sns.kdeplot(sascore_gen,  fill=True, label= str(np.round(np.mean(sascore_gen),2)), linewidth=3,alpha=.5) # 'Generated = ' + color='Green',
p2 = sns.kdeplot(sascore_real, fill=True, label= str(np.round(np.mean(sascore_real),2)), linewidth=3,alpha=.5) #'Experimental = ' +color='red', 
#p2.set(xlabel='Synthetic Acessibility Score', title='Distribution of SA Score for generated molecules')
p2.legend(bbox_to_anchor=(1.00, 1.00), loc='best',fontsize=12)
p2.set_xlabel('Synthetic acessibility score', fontsize=14)  # Adjust the font size as needed
p2.set_ylabel('Density', fontsize=14)  # Adjust the font size as needed
plt.tight_layout()
plt.savefig('./results/SAS_experimental_vs_generated.png',dpi = 600)
plt.show()